Our goal for this code is to left join the dataset of corporations since 1800 to the dataset of minority run businesses on the column Vendor_Formal_Name (the name of the business). 
Both datasets can be found here: 

https://data.ny.gov/Economic-Development/Active-Corporations-Beginning-1800/n9v6-gdp6/data

https://data.cityofnewyork.us/Business/M-WBE-LBE-and-EBE-Certified-Business-List/ci93-uc8s/data

In [1]:
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import inspect
import re
import datetime as dt

Let's initialize database with filename myDB in current directory

In [2]:
engine = sqlalchemy.create_engine('sqlite:///myDB.db')

Now let's upload our CSV in chunks (necessary since it is 2.8 million rows). First, define chunksize and indexes (so we don't overwrite the rows we just uploaded)

In [4]:
chunksize = 100000
j= 1
i = 0

Now we can load into the database. 

In [5]:
start = dt.datetime.now()
for df in pd.read_csv("C://Users//SEAB//Downloads//Active_Corporations___Beginning_1800.csv", chunksize = chunksize, iterator = True):
	df = df.rename(columns={c:c.replace(' ', '')  for c in df.columns})
	df.index +=j
	#notice that i is index for testing, in case you want to print which chunk you are at 
	i+=1
	print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, i*chunksize))
	#activeCo will be the name of your table
	df.to_sql('activeCo', engine, if_exists = 'append')
	#df.index is a range of the rows, so with df.index[-1], we are going to the last row number and adding 1
	j= df.index[-1] + 1

0 seconds: completed 100000 rows
10 seconds: completed 200000 rows
19 seconds: completed 300000 rows
29 seconds: completed 400000 rows
39 seconds: completed 500000 rows
48 seconds: completed 600000 rows
58 seconds: completed 700000 rows
68 seconds: completed 800000 rows
79 seconds: completed 900000 rows
90 seconds: completed 1000000 rows
107 seconds: completed 1100000 rows
119 seconds: completed 1200000 rows
129 seconds: completed 1300000 rows
141 seconds: completed 1400000 rows
154 seconds: completed 1500000 rows
165 seconds: completed 1600000 rows
176 seconds: completed 1700000 rows
187 seconds: completed 1800000 rows
197 seconds: completed 1900000 rows
207 seconds: completed 2000000 rows
217 seconds: completed 2100000 rows
226 seconds: completed 2200000 rows
235 seconds: completed 2300000 rows
244 seconds: completed 2400000 rows
253 seconds: completed 2500000 rows
262 seconds: completed 2600000 rows
271 seconds: completed 2700000 rows
280 seconds: completed 2800000 rows


Now let's start our connection so we can begin making queries 

In [6]:
conn = sqlite3.connect('myDB.db')
cur = conn.cursor()

Let's do some preprocessing of the data in order to improve our match rate

First, let's define a column that is CurrentEntityName but stripped of all spaces and punctuation. To this end, we will first define a function to strip spaces and punctuation. 

In [8]:
def cleanWord(word):
	removeThe = re.sub(r"The ","",word)
	cleaned = re.sub(r"(?i)\band\b|\bor\b|\bthe\b|\bof\b", "", removeThe)
	myStrippedName = re.sub(r"[.?!'-+&/, ]", "", cleaned)
	return myStrippedName

conn.create_function("cleanWord", 1, cleanWord)

Let's check our function: 

In [10]:
sql  ='SELECT cleanWord(CurrentEntityName) FROM activeCo AS Test LIMIT 3'
result = cur.execute(sql)
result.fetchall()

[('00:02:59LLC',), ('000LLC',), ('000MEADOWLANELLC',)]

Great, so it works as a function. Now, let's check that a column "CleanedEntityName" does not exist yet. 

In [15]:
result = conn.execute('SELECT * FROM activeCo LIMIT 1')
names = list(map(lambda x: x[0], result.description))
names

['index',
 'DOSID',
 'CurrentEntityName',
 'InitialDOSFilingDate',
 'County',
 'Jurisdiction',
 'EntityType',
 'DOSProcessName',
 'DOSProcessAddress1',
 'DOSProcessAddress2',
 'DOSProcessCity',
 'DOSProcessState',
 'DOSProcessZip',
 'CEOName',
 'CEOAddress1',
 'CEOAddress2',
 'CEOCity',
 'CEOState',
 'CEOZip',
 'RegisteredAgentName',
 'RegisteredAgentAddress1',
 'RegisteredAgentAddress2',
 'RegisteredAgentCity',
 'RegisteredAgentState',
 'RegisteredAgentZip',
 'LocationName',
 'LocationAddress1',
 'LocationAddress2',
 'LocationCity',
 'LocationState',
 'LocationZip']

So it doesn't exist yet. Let's create our column now. 

In [16]:
sql = 'ALTER TABLE activeCo ADD COLUMN CleanedEntityName TEXT;'
cur.execute(sql)
sql = 'UPDATE activeCo SET CleanedEntityName = cleanWord(CurrentEntityName)'
cur.execute(sql)

Let's create an index on this new column. *NOTE* We have to use index otherwise this query will be unbearably slow. 

In [17]:
sql = 'CREATE INDEX CleanedEntityIdx ON activeCo(CleanedEntityName COLLATE NOCASE)'
cur.execute(sql)

Now let's test that the index is being used. 

In [18]:
params = ('Alpha%',)
sql = 'EXPLAIN QUERY PLAN SELECT * FROM activeCo WHERE CleanedEntityName LIKE ?;'
cur.execute(sql, params)
cur.fetchall()

[(3,
  0,
  0,
  'SEARCH TABLE activeCo USING INDEX CleanedEntityIdx (CleanedEntityName>? AND CleanedEntityName<?)')]

Great, so our index is working. Now let's read in our other dataset. 

In [19]:
my_df = pd.read_csv("C://Users//SEAB//Downloads//M_WBE__LBE__and_EBE_Certified_Business_List.csv")
numberRows = my_df.shape[0]

Let's read in some suffixes that we'd like to drop and create a new list of business names based off this. 

In [20]:
my_list= ['LLC', 'CORP',
          'INC', 'CORPORATION',
          'PLLC', 'PC',
          'LLP', 'CO', 'COMPANY', 'LTD']
myNames = [None]*numberRows

for index, row in my_df.iterrows():
        temp1= row['Vendor_Formal_Name'].rsplit(',', 1)
        temp2 = row['Vendor_Formal_Name'].rsplit(' ',1)
        if len(temp1) > 1 and re.sub(r"[,.?!'-+&/ ]", "", temp1[1]).upper() in my_list:
                myNames[index] = temp1[0]
        elif len(temp2) > 1 and re.sub(r"[,.?!'-+&/ ]", "", temp2[1]).upper() in my_list:
                        myNames[index] = temp2[0]
        else:
                myNames[index]=row['Vendor_Formal_Name']

Now let's get our query results. This should take less than a minute. 

In [21]:
myresponses= [None]*numberRows
start = dt.datetime.now()
for index, name in enumerate(myNames):
        sql = 'SELECT * from activeCo WHERE CleanedEntityName LIKE ?;'
        cleanName = cleanWord(name)
        params =(cleanName + '%',)
        response = cur.execute(sql, params)
        tables = response.fetchall()
        myresponses[index] = tables
print('{} seconds: completed getting query results'.format((dt.datetime.now() - start).seconds))


2 seconds: completed getting query results


Let's see how percentage of nonempty results we got: 

In [25]:
str(((numberRows - myresponses.count([]))/numberRows)*100) + '%'

'87.31236989153062%'

It's okay. We can do better perhaps with some more sophisticated searching methods but we also have to consider the tradeoff between complexity and completeness. 

Let's get our column names again so we can write to dataframe

In [26]:
result = conn.execute('SELECT * FROM activeCo LIMIT 1')
names = list(map(lambda x: x[0], result.description))
numberColumns = len(names)

Now let's create new list of tuples for conversion to dataframe 

In [27]:
myNewResponses = [response[0] if response != [] else tuple([None]*numberColumns) for response in myresponses]
new_df = pd.DataFrame(myNewResponses, columns = names)
temp = pd.concat([my_df, new_df], axis = 1)
temp = temp.drop(['CleanedEntityName'], axis = 1)
cols = temp.columns.tolist()

Finally, let's move CurrentEntityName to the front of table in order for more convenient comparison between matched values. 

In [29]:
cols.insert(0, cols.pop(cols.index('CurrentEntityName')))
final_df = temp.reindex(columns = cols)
final_df.to_csv("MyJoinedData.csv")

In [30]:
final_df.head()

,CurrentEntityName,Vendor_Formal_Name,Vendor_DBA,Contact_Name,telephone,fax,Email,certification,Cert_Renewal_Date,Ethnicity,...,RegisteredAgentAddress2,RegisteredAgentCity,RegisteredAgentState,RegisteredAgentZip,LocationName,LocationAddress1,LocationAddress2,LocationCity,LocationState,LocationZip
0,BRYCE CONSTRUCTION CORPORATION,Bryce Construction Corp.,NaN,Patrick Nwokeji,6465235353,9736786112,bryce.constructioncorp@gmail.com,MBE,06/30/2024,BLACK,...,None,None,None,None,BRYCE CONSTRUCTION CORPORATION,450 EAST HAINES ST,None,PHILADELPHIA,PENNSYLVANIA,19144
1,ICOMMERCIALIZE LLC,iCommercialize LLC,NaN,Beth Hirschhorn,9177348658,NaN,beth.hirschhorn@icommercialize.co,WBE,NaN,NON-MINORITY,...,None,None,None,None,None,None,None,None,None,None
2,SLICE WIRELESS SOLUTIONS LLC,Slice Wireless Solutions LLC,NaN,Aleksandr Yakubov,2128686900,6464488238,NaN,WBE,NaN,NON-MINORITY,...,None,None,None,None,None,None,None,None,None,None
3,ALPHA AND OMEGA CONSULTING GROUP INC.,"Alpha and Omega Consulting Group, Inc",NaN,Cinque Braithwaite,9172840028,NaN,mpericles1@yahoo.com,"MBE,WBE",02/27/2020;02/27/2020,BLACK,...,None,CAMBRIA HEIGHTS,NEW YORK,11411,None,None,None,None,None,None
4,"BAY RIDGE SECURITY SERVICE, INC.",Bay Ridge Security Service Inc.,NaN,Anthony La Bella,7182382727,7188363474,nl@bayridgesecurity.com,WBE,NaN,NON-MINORITY,...,None,None,None,None,"BAY RIDGE SECURITY SERVICE, INC.",110 BAY RIDGE AVENUE,None,BROOKLYN,NEW YORK,11220
